In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2023-04-11 15:51:37.006043: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 15:51:37.082009: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 15:51:54.989563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print('Libraries loaded successfully')

Libraries loaded successfully


In [3]:
def preprocessing(fasta_file, temp_file):
    l = 1000
    fp = open(temp_file, 'w')
    
    with open (fasta_file, 'r') as file:
        seq=[]
        seq_id=[]
        for line in file:
            if line.startswith('>'):
                seq_id.append(line.strip('>').strip('\n'))
            else:
                seq.append(line.strip('\n'))
            
    for j, sq in enumerate(seq):   
        if len(sq) <= 1000:
            fp.write('>%s\n'%(seq_id[j]))
            fp.write('%s\n'%(seq[j]))
        else:
            for i in range(0, len(seq)-l+1, 100):
                new_seq_id = '%s_SEPARATED_SEQUENCE_(%s_%s)' % (seq_id[j], i+1, i+l+1)
                new_seq = sq[i:i+l]
                fp.write('>%s\n'%(new_seq_id))
                fp.write('%s\n'%(new_seq))
    
    #input_handle.close()
    fp.close()
    return 

In [4]:
def fill_aa(seq):
    fill_aa_cnt = 1000 - len(seq)
    add_aa_seq = '_' * fill_aa_cnt
    new_seq = seq + add_aa_seq
    return new_seq

In [5]:
def score_info():
    aa_list = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X', '_']
    aa_score_info = {}
    for aa in aa_list:
        for aa2 in aa_list:
            if aa == aa2:
                aa_score_info[(aa, aa2)] = 1.0
                aa_score_info[(aa2, aa)] = 1.0
            else:
                aa_score_info[(aa, aa2)] = 0.0
                aa_score_info[(aa2, aa)] = 0.0
    return aa_score_info

In [6]:
def one_hot_encoding(seq, aa_score_info):
    data = np.zeros((1000, 21), dtype=np.float32)
    aa_list = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X']
    for i, aa in enumerate(seq):
        for j, aa2 in enumerate(aa_list):
            data[i, j] = aa_score_info[aa, aa2]
    return data

In [7]:
counter=0
def create_new_file(file):
    global counter
    counter += 1
    filename = f"{file}/{file}_{counter}.txt"
    return open(filename, 'w')


In [8]:
def run_one_hot_encoding(fasta_file2, temp_file):
    aa_score_info = score_info()
    
    os.makedirs(temp_file)
    
    fp = create_new_file(temp_file)
    
    feature_names = ['Feature%s'%(i) for i in range(1, 21001)] 
    
    fp.write('%s\n'%(','.join(['ID']+feature_names)))
    
    with open (fasta_file2, 'r') as file:
        seq=[]
        seq_id=[]
        for line in file:
            if line.startswith('>'):
                seq_id.append(line.strip('>').strip('\n'))
            else:
                seq.append(line.strip('\n'))
                
                
    for j, sq in enumerate(seq):
        try:
            if len(sq) >= 10 and len(sq) <= 1000:
                if len(sq) < 1000:
                    sq = fill_aa(sq)
                encoded_vector = one_hot_encoding(sq, aa_score_info)
                flatten_encoded_vector = encoded_vector.flatten()
                flatten_encoded_vector_str = [str(each_val) for each_val in flatten_encoded_vector]
                fp.write('%s\n'%(','.join([seq_id[j]]+flatten_encoded_vector_str)))
                
                # Check if the current iteration is a multiple of 1000
                if (j + 1) % 1000 == 0:
                    # If it is, close the current file and create a new one
                    fp.close()
                    fp = create_new_file(temp_file)
                    fp.write('%s\n'%(','.join(['ID']+feature_names)))
            else:
                pass
        except:
            pass
    fp.close()

In [9]:
def create_df(df):
    seq_ids = list(df.index)
    X_temp = df.values
    new_X = []
    for i in range(len(X_temp)):
        temp = np.reshape(X_temp[i], (1000, 21))
        new_X.append(temp)
    
    X = np.asarray(new_X)
    X = X.reshape(X.shape[0], 1000, 21, 1)
    #print('numpy array created')
    return X

In [10]:
#creating dataset
def create_dataset(fasta, tmp_file):
    preprocessing(fasta, 'tmp.txt')
    print('preprocessing done')
    run_one_hot_encoding('tmp.txt', tmp_file)
    print('one hot encoding done')
    return

In [11]:
def create_array(temp_file1, val1):
    
    file_list = os.listdir(temp_file1)
    file_list.sort()
    file_list
    
    X=np.empty((0, 1000, 21, 1))
    y=np.empty((0,2))
    
    for file in file_list:
        file_path=temp_file1+'/'+file
        temp_df = pd.read_csv(file_path, index_col=0)
        #print('temp df loaded')
        X_new=create_df(temp_df)
        X=np.concatenate((X, X_new), axis=0)
        #print('X created')
        y_new=np.full(len(temp_df.index), val1)
        y_new= to_categorical(y_new, 2)
        y=np.concatenate((y, y_new), axis=0)
    
    return X, y
    print('done')

In [12]:
print('functions defined')

functions defined


In [13]:
create_dataset('enzyme_90.fasta', 'tmp_enz')
print('Enzyme dataset loaded')

preprocessing done
one hot encoding done
Enzyme dataset loaded


In [14]:
os.remove('tmp.txt')
print('removed tmp seq file')

removed tmp seq file


In [15]:
create_dataset('non_enzyme_90.fasta', 'tmp_non_enz')
print('non enzyme dataset loaded')

preprocessing done
one hot encoding done
non enzyme dataset loaded


In [16]:
os.remove('tmp.txt')
print('removed tmp seq file')

removed tmp seq file


In [13]:
X, y=create_array('tmp_enz',1)
print('enzyme dataset loaded')

enzyme dataset loaded


In [14]:
print('saving numpy files')
np.save('X_1',X)
np.save('y_1',y)
print('numpy files saved')

saving numpy files
numpy files saved


In [15]:
#setting X, y to zero to save memory
X,y=[],[]

print('clearing memory')

clearing memory


In [16]:
X, y=create_array('tmp_non_enz',0)
print('non enzyme dataset loaded')

non enzyme dataset loaded


In [17]:
print('saving numpy files')
np.save('X_2',X)
np.save('y_2',y)
print('numpy files saved')

saving numpy files
numpy files saved


In [18]:
#setting X, y to zero to save memory
X,y=[],[]

print('clearing memory')

clearing memory


In [2]:
#creating memory maps
import numpy as np
print('creating memory maps')
X_1 = np.lib.format.open_memmap('X_1.npy', mode='r', dtype='float32')
print('memory map for X created')
y_1 = np.lib.format.open_memmap('y_1.npy', mode='r', dtype='float32')
print('memory map for y created')
X_2 = np.lib.format.open_memmap('X_2.npy', mode='r', dtype='float32')
print('memory map for X created')
y_2 = np.lib.format.open_memmap('y_2.npy', mode='r', dtype='float32')
print('memory map for y created')
print('memory map created')

creating memory maps
memory map for X created
memory map for y created
memory map for X created
memory map for y created
memory map created


In [30]:
X_1.shape[0]+X_2.shape[0]

393850

In [43]:
X=np.lib.format.open_memmap('X.npy', dtype=np.float32, mode='w+',shape=(0,1000,21,1))

In [44]:
X=np.concatenate((X, X_1), axis=0)


In [45]:
X.shape

(212146, 1000, 21, 1)

In [46]:
X=np.concatenate((X, X_2), axis=0)


In [47]:
np.save('X.npy', X)

In [54]:
X=[]

In [3]:
y=np.concatenate((y_1, y_2), axis=0)
np.save('y.npy', y)

In [48]:
import dask.array as da

In [53]:
#using dask
import dask.diagnostics

with dask.diagnostics.ProgressBar():
    y_1=da.from_npy_stack('Ravi/CNN/batched/99/y_1.npy')
    y_2=da.from_npy_stack('y_2.npy')
    
    y= da.concatenate([y_1,y_2], axis=0)
    
    da.to_npy_stack('y', y)

FileNotFoundError: [Errno 2] No such file or directory: 'Ravi/CNN/batched/99/y_1.npy/info'

In [2]:
X= np.lib.format.open_memmap('X.npy', mode='r', dtype='float32')
y= np.lib.format.open_memmap('y.npy', mode='r', dtype='float32')


In [3]:
print('starting split')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=11)
print('train and validation split done')

starting split
train and validation split done


In [4]:
print('saving split numpy files')
np.save('X_train_11',X_train)
np.save('y_train_11',y_train)
np.save('X_val_11',X_val)
np.save('y_val_11',y_val)

print('numpy files saved')

saving split numpy files
numpy files saved


In [5]:
X_train, X_val, y_train, y_val =[],[],[],[]
print('clearing memory')

clearing memory


In [ ]:
print('Files for training ready')